In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import numpy as np
import statsmodels.api as sm
import statsmodels.api as smf
import re
from scipy.stats import poisson
from sklearn.linear_model import LinearRegression
import html5lib


In [2]:
def SerieA():
    url0="https://fbref.com/es/comps/11/Estadisticas-de-Serie-A"
    urlpartido0="https://fbref.com/es/comps/11/horario/Resultados-y-partidos-en-Serie-A"
    url1="https://fbref.com/es/comps/11/2022-2023/Estadisticas-2022-2023-Serie-A"
    urlpartido1="https://fbref.com/es/comps/11/2022-2023/horario/Marcadores-y-partidos-de-2022-2023-Serie-A"
    return url0,urlpartido0,url1,urlpartido1
def PremierLeague():
    url0="https://fbref.com/es/comps/9/Estadisticas-de-Premier-League"
    urlpartido0="https://fbref.com/es/comps/9/horario/Resultados-y-partidos-en-Premier-League"
    url1="https://fbref.com/es/comps/9/2022-2023/Estadisticas-2022-2023-Premier-League"
    urlpartido1="https://fbref.com/es/comps/9/2022-2023/horario/Marcadores-y-partidos-de-2022-2023-Premier-League"
    return url0,urlpartido0,url1,urlpartido1
def Bundesliga():
    url0="https://fbref.com/es/comps/20/Estadisticas-de-Bundesliga"
    urlpartido0="https://fbref.com/es/comps/20/horario/Resultados-y-partidos-en-Bundesliga"
    url1="https://fbref.com/es/comps/20/2022-2023/Estadisticas-2022-2023-Bundesliga"
    urlpartido1="https://fbref.com/es/comps/20/2022-2023/horario/Marcadores-y-partidos-de-2022-2023-Bundesliga"
    return url0,urlpartido0,url1,urlpartido1
def LeagueWomen():
    url0="https://fbref.com/es/comps/196/Estadisticas-de-A-League-Women"
    urlpartido0="https://fbref.com/es/comps/196/horario/Resultados-y-partidos-en-A-League-Women"
    url1="https://fbref.com/es/comps/196/2022-2023/Estadisticas-2022-2023-A-League-Women"
    urlpartido1="https://fbref.com/es/comps/196/2022-2023/horario/Marcadores-y-partidos-de-2022-2023-A-League-Women"
    return url0,urlpartido0,url1,urlpartido1
def Eredivisie():
    url0="https://fbref.com/es/comps/23/Estadisticas-de-Eredivisie"
    urlpartido0="https://fbref.com/es/comps/23/horario/Resultados-y-partidos-en-Eredivisie"
    url1="https://fbref.com/es/comps/23/2022-2023/Estadisticas-2022-2023-Eredivisie"
    urlpartido1="https://fbref.com/es/comps/23/2022-2023/horario/Marcadores-y-partidos-de-2022-2023-Eredivisie"
    return url0,urlpartido0,url1,urlpartido1
def menu(opcion):
    if opcion=="1":
        url0,urlpartido0,url1,urlpartido1=SerieA()
        return url0,urlpartido0,url1,urlpartido1
    elif opcion=="2":
        url0,urlpartido0,url1,urlpartido1=PremierLeague()
        return url0,urlpartido0,url1,urlpartido1
    elif opcion=="3":
        url0,urlpartido0,url1,urlpartido1=Bundesliga()
        return url0,urlpartido0,url1,urlpartido1
    elif opcion=="4":
        url0,urlpartido0,url1,urlpartido1=LeagueWomen()
        return url0,urlpartido0,url1,urlpartido1
    elif opcion=="5":
        url0,urlpartido0,url1,urlpartido1=Eredivisie()
        return url0,urlpartido0,url1,urlpartido1
print("1.SerieA")
print("2.PremierLeague")
print("3.Bundesliga")
print("4.LeagueWomen")
print("5.Eredivisie")    

In [353]:
url0,urlpartido0,url1,urlpartido1=menu("2")
datosActual=pd.read_html(url0)
datosPasada=pd.read_html(url1)
datosxpartido1=pd.read_html(urlpartido0)
datosxpartido2=pd.read_html(urlpartido1)
partidoxActual=pd.DataFrame(datosxpartido1[0])#actual
partidoxActual["Sem."]=partidoxActual["Sem."]+38
partidoxPasada=pd.DataFrame(datosxpartido2[0])#data pasada
partido=pd.DataFrame(datosxpartido1[0])#actual


In [354]:
def GEP(data):
    almacenamiento = []

    for i in range(len(data)):
        g = e = p = 0
        GolesL = data["GolLoc"].iloc[i]
        GolesV = data["GolVis"].iloc[i]

        if GolesL > GolesV:
            g += 1
        elif GolesL == GolesV:
            e += 1
        elif GolesL < GolesV:
            p += 1

        almacenamiento.append({
            "G": g,
            "E": e,
            "P": p
        })

    # Crear un DataFrame con los resultados de almacenamiento
    resultados_df = pd.DataFrame(almacenamiento)

    # Unir el DataFrame de resultados con el DataFrame original
    data = pd.concat([data, resultados_df], axis=1)

    return data
def Extractor(Data_General, Fin, Inicio):
    # Crear una copia explícita del DataFrame
    Data = Data_General.copy()

    filtro = (Data["Sem."] <= Fin) & (Data["Sem."] >= Inicio)
    Data = Data[filtro]
    
    # Crear nuevas columnas sin afectar el DataFrame original
    Data[['GolLoc', 'GolVis']] = Data['Marcador'].str.split('–', expand=True)
    # Entero
    Data['GolLoc'] = Data['GolLoc'].astype(int)
    Data['GolVis'] = Data['GolVis'].astype(int)
    # Eliminar la columna original 'Marcador'
    Data = Data.drop('Marcador', axis=1).reset_index(drop=True)
    return Data
def FPC ( Data ) :
    #Equipos
    EquipoLocal=Data.groupby("Local")
    EquipoVisitante=Data.groupby("Visitante")
    
    TeamLocal = EquipoLocal["Local"].unique().tolist()
    Team_vistante = EquipoVisitante["Visitante"].unique().tolist()
    #conteo de partidos
    conteo_partidos_local = EquipoLocal["Local"].agg(len).values
    conteo_partidos_visitante = EquipoVisitante["Visitante"].agg(len).values
    #Tablas de conteo Local
    TABLA_FPC2_LOCAL = pd.DataFrame(columns=["TEAM","PJ"])
    TABLA_FPC2_VISITANTE = pd.DataFrame(columns=["TEAM","PJ"])
    TABLA_FPC2_LOCAL.TEAM = TeamLocal    
    TABLA_FPC2_LOCAL.PJ = conteo_partidos_local
    #Tablas de conteo Visitante
    TABLA_FPC2_VISITANTE = pd.DataFrame(columns=["TEAM","PJ"])
    TABLA_FPC2_VISITANTE.TEAM = Team_vistante   
    TABLA_FPC2_VISITANTE.PJ = conteo_partidos_visitante
    TABLA_FPC_LOCAL = pd.DataFrame(columns=("Equipo","PGL","PEL","PPL","GFL","GCL"))
    TABLA_FPC_LOCAL.Equipo = TeamLocal
    TABLA_FPC_LOCAL.PGL = EquipoLocal.G.sum().values / TABLA_FPC2_LOCAL.PJ.values 
    TABLA_FPC_LOCAL.PEL = EquipoLocal.E.sum().values / TABLA_FPC2_LOCAL.PJ.values 
    TABLA_FPC_LOCAL.PPL = EquipoLocal.P.sum().values / TABLA_FPC2_LOCAL.PJ.values 
    TABLA_FPC_LOCAL.GFL = EquipoLocal.GolLoc.sum().values / TABLA_FPC2_LOCAL.PJ.values 
    TABLA_FPC_LOCAL.GCL = EquipoLocal.GolVis.sum().values / TABLA_FPC2_LOCAL.PJ.values 
    TABLA_FPC_VISITANTE = pd.DataFrame(columns=("Equipo","PGV","PEV","PPV","GFV","GCV"))
    TABLA_FPC_VISITANTE.Equipo = Team_vistante
    TABLA_FPC_VISITANTE.PGV = EquipoVisitante.G.sum().values / TABLA_FPC2_VISITANTE.PJ.values 
    TABLA_FPC_VISITANTE.PEV = EquipoVisitante.E.sum().values / TABLA_FPC2_VISITANTE.PJ.values  
    TABLA_FPC_VISITANTE.PPV = EquipoVisitante.P.sum().values / TABLA_FPC2_VISITANTE.PJ.values 
    TABLA_FPC_VISITANTE.GFV = EquipoVisitante.GolLoc.sum().values / TABLA_FPC2_VISITANTE.PJ.values 
    TABLA_FPC_VISITANTE.GCV = EquipoVisitante.GolVis.sum().values / TABLA_FPC2_VISITANTE.PJ.values
    #Elimina las diferencias de las tablas para su igualdad
    DiferenciasLocal = np.setdiff1d(TABLA_FPC_LOCAL["Equipo"],TABLA_FPC_VISITANTE["Equipo"])
    DiferenciasVisitante = np.setdiff1d(TABLA_FPC_VISITANTE["Equipo"],TABLA_FPC_LOCAL["Equipo"])
    TABLA_FPC_LOCAL = (TABLA_FPC_LOCAL[~TABLA_FPC_LOCAL["Equipo"].isin(DiferenciasLocal)]).reset_index(drop=True)
    TABLA_FPC_VISITANTE = (TABLA_FPC_VISITANTE[~TABLA_FPC_VISITANTE["Equipo"].isin(DiferenciasVisitante)]).reset_index(drop = True)
    TABLA_FPC_GOL = pd.DataFrame(columns=("GFT","GCT"))
    TABLA_FPC_GOL.GFT = ( TABLA_FPC_LOCAL.GFL + TABLA_FPC_VISITANTE.GFV ) / 2
    TABLA_FPC_GOL.GCT = ( TABLA_FPC_LOCAL.GCL + TABLA_FPC_VISITANTE.GCV ) / 2
    TABLA_FPC = pd.DataFrame(columns=("Equipo","PGL","PEL","PPL","GFL","GCL","PGV","PEV","PPV","GFV","GCV","GFT","GCT"))
    TABLA_FPC.Equipo = TABLA_FPC_LOCAL.Equipo
    TABLA_FPC.PGL = TABLA_FPC_LOCAL.PGL
    TABLA_FPC.PEL = TABLA_FPC_LOCAL.PEL
    TABLA_FPC.PPL = TABLA_FPC_LOCAL.PPL
    TABLA_FPC.GFL = TABLA_FPC_LOCAL.GFL
    TABLA_FPC.GCL = TABLA_FPC_LOCAL.GCL
    TABLA_FPC.PGV = TABLA_FPC_VISITANTE.PGV
    TABLA_FPC.PEV = TABLA_FPC_VISITANTE.PEV
    TABLA_FPC.PPV = TABLA_FPC_VISITANTE.PPV
    TABLA_FPC.GFV = TABLA_FPC_VISITANTE.GFV
    TABLA_FPC.GCV = TABLA_FPC_VISITANTE.GCV
    TABLA_FPC.GFT = TABLA_FPC_GOL.GFT
    TABLA_FPC.GCT = TABLA_FPC_GOL.GCT

    return TABLA_FPC
def StatusInitial(TABLA_FPC):
    TABLA_FUERZAS=pd.DataFrame(columns=["EQUIPO",'FOFENSIVAA','FOFENSIVAB','FDEFENSIVAA','FDEFENSIVAB','GCT',"GFT"])

#organizar tablas de fuerzas

    TABLA_FUERZAS. EQUIPO =  TABLA_FPC.Equipo
    TABLA_FUERZAS. FOFENSIVAA = (  TABLA_FPC.GFL / TABLA_FPC.GFT )
    TABLA_FUERZAS. FOFENSIVAB = (  TABLA_FPC.GFV / TABLA_FPC.GCT )
    TABLA_FUERZAS. FDEFENSIVAA = ( TABLA_FPC.GCL / TABLA_FPC.GCT )
    TABLA_FUERZAS. FDEFENSIVAB = ( TABLA_FPC.GCV / TABLA_FPC.GFT )
    TABLA_FUERZAS. GCT =( TABLA_FPC.GCT)
    TABLA_FUERZAS. GFT =( TABLA_FPC.GFT)
    return TABLA_FUERZAS
def ConcatTables (TABLA_FUERZAS_INITIAL,TABLA_FUERZAS_FINAL,round,TEAM_ACTUAL,i):
    round=38
    TABLA_FUERZAS = {"EQUIPO": [], "FOFENSIVAA": [], "FOFENSIVAB": [] , "FDEFENSIVAA": [], "FDEFENSIVAB": [],"GCT": [],"GFT": []}
    #Operaciones Matematicas
    Constant = (1*100)/ round
    Data_Initial = TABLA_FUERZAS_INITIAL.select_dtypes(include='number')
    Data_Final = TABLA_FUERZAS_FINAL.select_dtypes(include='number')
    #Reemplazar NaN con ceros
    Data_Initial.fillna(0 , inplace = True)
    Data_Final.fillna(0 , inplace = True)
    # Reemplazar inf con 0
    Data_Initial.replace([np.inf , -np.inf] , 0 ,inplace = True)
    Data_Final.replace([np.inf , -np.inf] , 0 ,inplace = True)
    #Operacioones matematicas
    Data_Initial = (Data_Initial * (Constant*i)) / 100
    Data_Final = (Data_Final *  (Constant*(round-i))) / 100#(i-round) represeta el decremento de ronda
    data = []
    for i in range(len(TEAM_ACTUAL)):
        Team = TEAM_ACTUAL[i]
        filtro1 = TABLA_FUERZAS_INITIAL["EQUIPO"] == Team
        filtro2 = TABLA_FUERZAS_FINAL["EQUIPO"] == Team
        Data1 = Data_Initial[filtro1]
        Data2 = Data_Final[filtro2]
    
        if Data1.empty : 
            Data1 = Data2
        elif Data2.empty:
            Data2 = Data1 
        #TABLA_FUERZAS 
        TABLA_FUERZAS["EQUIPO"].append(Team)
        TABLA_FUERZAS["FOFENSIVAA"].append((Data1["FOFENSIVAA"].iloc[0] + Data2["FOFENSIVAA"].iloc[0]) / 2)
        TABLA_FUERZAS["FOFENSIVAB"].append((Data1["FOFENSIVAB"].iloc[0] + Data2["FOFENSIVAB"].iloc[0]) / 2)
        TABLA_FUERZAS["FDEFENSIVAA"].append((Data1["FDEFENSIVAA"].iloc[0] + Data2["FDEFENSIVAA"].iloc[0]) / 2)
        TABLA_FUERZAS["FDEFENSIVAB"].append((Data1["FDEFENSIVAB"].iloc[0] + Data2["FDEFENSIVAB"].iloc[0]) / 2)
        TABLA_FUERZAS["GCT"].append((Data1["GCT"].iloc[0] + Data2["GCT"].iloc[0]) / 2)
        TABLA_FUERZAS["GFT"].append((Data1["GFT"].iloc[0] + Data2["GFT"].iloc[0]) / 2)
    DF_TABLA_FUERZAS = pd.DataFrame(TABLA_FUERZAS)

    return DF_TABLA_FUERZA
def Tabla_Juegos(Tabla_fuerzas, filtered_rows, Sem):
    # Create an empty list to store results
    TablaJuegos = {"Sem.": [], "Local": [], "Visitante": [], "GolLoc": [], "GolVis": []}

    # Loop through filtered_rows to calculate game results
    for i in range(len(filtered_rows)):
        Team1 = filtered_rows["Local"].iloc[i]
        Team2 = filtered_rows["Visitante"].iloc[i]

        filtro1 = Tabla_fuerzas["EQUIPO"] == Team1
        filtro2 = Tabla_fuerzas["EQUIPO"] == Team2
        data1 = Tabla_fuerzas[filtro1]
        data2 = Tabla_fuerzas[filtro2]

        fuerzaofecivaALocal = data1["FOFENSIVAA"].iloc[0] if not data1.empty else 0
        fuerzadefencivaBLocal = data1["FDEFENSIVAB"].iloc[0] if not data1.empty else 0
        gctotalLocal = data1["GCT"].iloc[0] if not data1.empty else 0

        fuerzaofecivaAVisitante = data2["FOFENSIVAA"].iloc[0] if not data2.empty else 0
        fuerzadefencivaBVisitante = data2["FDEFENSIVAB"].iloc[0] if not data2.empty else 0
        gctotalVisitante = data2["GCT"].iloc[0] if not data2.empty else 0

        # Calculate goals and append to the dictionary
        TablaJuegos["Sem."].append(Sem)
        TablaJuegos["Local"].append(Team1)
        TablaJuegos["Visitante"].append(Team2)

        try:
            GolLoc = round((fuerzaofecivaALocal * fuerzadefencivaBVisitante * gctotalLocal))
            TablaJuegos["GolLoc"].append(GolLoc if GolLoc >= 0 else 0)
        except ValueError as e:
            print(f"Error calculando GolLoc para {Team1} vs {Team2}: {e}")
            TablaJuegos["GolLoc"].append(0)

        try:
            GolVis = round((fuerzaofecivaAVisitante * fuerzadefencivaBLocal * gctotalVisitante))
            TablaJuegos["GolVis"].append(GolVis if GolVis >= 0 else 0)
        except ValueError as e:
            print(f"Error calculando GolVis para {Team1} vs {Team2}: {e}")
            TablaJuegos["GolVis"].append(0)
    df_Tabla_Juegos = pd.DataFrame(TablaJuegos)
    return  df_Tabla_Juegos 
        

In [511]:
Predict = pd.DataFrame()
for i in range(39,58):
    DATA_GENERAL = pd.concat([partidoxPasada,partidoxActual]).dropna(subset="Marcador").drop(["Día","Fecha","Hora","xG","xG.1","Sedes","Árbitro","Informe del partido","Notas","Asistencia"],axis=1)
    TEAM_ACTUAL = partidoxActual["Local"].dropna().unique().tolist()
    TEAM_PASADA = partidoxPasada["Local"].dropna().unique().tolist()
    DATA_GENERAL_ULTIMOS =Extractor(DATA_GENERAL,i,(i-10))
    DATA_GENERAL =Extractor(DATA_GENERAL,i,1)
    DATA_GENERAL = GEP(DATA_GENERAL)
    DATA_GENERAL_ULTIMOS = GEP(DATA_GENERAL_ULTIMOS)
    TABLA_FPC= FPC (DATA_GENERAL)
    TABLA_FPC_ULTIMOS= FPC (DATA_GENERAL_ULTIMOS)
    TABLA_FUERZAS_FINAL = StatusInitial(TABLA_FPC)
    TABLA_FUERZAS_ULTIMOS = StatusInitial(TABLA_FPC_ULTIMOS)
    filtered_rows = partidoxActual.loc[partidoxActual['Sem.'] == i].copy()
    filtered_rows=filtered_rows.drop(["Día","Fecha","Hora","xG","xG.1","Asistencia","Sedes","Árbitro","Informe del partido","Notas","Marcador"],axis=1)
    iguales=TABLA_FUERZAS_FINAL.equals(TABLA_FPC_ULTIMOS)
    Diferenciasfinal = np.setdiff1d(TABLA_FUERZAS_FINAL["EQUIPO"],TABLA_FUERZAS_ULTIMOS["EQUIPO"])
    TABLA_FUERZAS_GENERAL= (TABLA_FUERZAS_FINAL[~TABLA_FUERZAS_FINAL["EQUIPO"].isin(Diferenciasfinal)]).reset_index(drop=True)
    TABLA_FUERZAS = (TABLA_FUERZAS_GENERAL.drop(["EQUIPO"], axis=1) + (TABLA_FUERZAS_ULTIMOS.drop(["EQUIPO"], axis=1))) / 2
    TABLA_FUERZAS["EQUIPO"] = TABLA_FUERZAS_FINAL.EQUIPO
    TABLA_JUEGOS = Tabla_Juegos(TABLA_FUERZAS,filtered_rows,i)
    Predict = pd.concat([Predict,TABLA_JUEGOS])
Predict = GEP(Predict.reset_index(drop = True))

In [503]:
filtro = (DATA_GENERAL["Sem."] >= 39) & (DATA_GENERAL["Sem."] <= 58)
datos = DATA_GENERAL[filtro]

In [504]:
DATA={"Sem.":[] ,"TrueGolLoc":[] , "FalseGolLoc":[] , "TrueGolVis":[] , "FalseGolVis":[] , "TrueEstado":[] , "FalseEstado":[]}
for i in range(39 , 58):
    filtro1 = DATA_GENERAL["Sem."] == i
    filtro2 = Predict["Sem."] == i
    y_true = DATA_GENERAL[ filtro1 ].reset_index(drop=True)
    y_predict = Predict[ filtro2 ].reset_index(drop=True)
    #print(y_true)
    #print(y_predict)

    # Comparar si los DataFrames son iguales
    son_identicos = y_true["Local"].equals(y_predict["Local"])
    Diferencias = np.setdiff1d(np.array(y_predict["Local"]),np.array(y_true["Local"]))
    if not son_identicos:
        y_predict = y_predict[~y_predict["Local"].isin(Diferencias)]
    
    diff_columns = ["Local","Visitante",'GolLoc', 'GolVis',"G","E","P"]
    diff_values = y_true[diff_columns] == y_predict[diff_columns]
    #print(diff_values)
    GolLoc_Count = diff_values["GolLoc"].value_counts().values
    GolVis_Count = diff_values["GolVis"].value_counts().values
    filas_condicion = (diff_values['G'] & diff_values['E'] & diff_values['P']).value_counts().values
    DATA["Sem."].append(i)
    try:
        DATA["TrueGolLoc"].append(GolLoc_Count[1])
    except IndexError :
        DATA["TrueGolLoc"].append(0)
    try:    
        DATA["FalseGolLoc"].append(GolLoc_Count[0])
    except IndexError :
        DATA["FalseGolLoc"].append(0)
    try:       
        DATA["TrueGolVis"].append(GolVis_Count[1])
    except IndexError :
        DATA["TrueGolVis"].append(0)
    try:    
        DATA["FalseGolVis"].append(GolVis_Count[0])
    except IndexError:
        DATA["FalseGolVis"].append(0)
    try:        
        DATA["TrueEstado"].append(GolVis_Count[1])
    except IndexError :
        DATA["TrueEstado"].append(0)
    try:
        DATA["FalseEstado"].append(GolVis_Count[0])
    except IndexError :
        DATA["FalseEstado"].append(0)

In [505]:
Resultado = pd.DataFrame(DATA)
Resultado

,Sem.,TrueGolLoc,FalseGolLoc,TrueGolVis,FalseGolVis,TrueEstado,FalseEstado
0,39,3,7,4,6,4,6
1,40,1,9,2,8,2,8
2,41,4,6,4,6,4,6
3,42,1,9,3,7,3,7
4,43,4,6,4,6,4,6
5,44,3,7,4,6,4,6
6,45,2,8,4,6,4,6
7,46,2,8,5,5,5,5
8,47,2,8,2,8,2,8
9,48,2,8,3,7,3,7


In [517]:
posibilidad = Resultado["TrueEstado"].sum() / 180
posibilidad 

0.35555555555555557